<a href="https://colab.research.google.com/github/lsh950919/Portfolio/blob/main/NIPA_Competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2020 인공지능 문제해결 경진대회 예선과제
정보통신산업진흥원 (National IT Industry Promotion Agency or NIPA) 에서 주최한 경진대회의 예선 과제에서 사용된 코드입니다.

나뭇잎 사진을 학습하여 나뭇잎의 종류와 나뭇잎이 감염된 질병을 분류하는 과제였으며, 16000장의 train셋과 4000장의 test셋이 주어졌고 예측값의 F1 score가 평가 기준이었습니다

사용된 모델은 배치 정규화가 포함된 VGG-19 모델이며, 최종 F1 score는 0.995였습니다

https://ai-korea.kr/info/contestPost.do

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/NIPA Competition

/content/drive/My Drive/Colab Notebooks/NIPA Competition


In [ ]:
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.6-cp36-cp36m-linux_x86_64.whl

In [ ]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from PIL import Image
import random

In [ ]:
train_tsv = pd.read_csv('train/train.tsv', sep = '\t', header = None, names = ['파일', '종', '병'])
train_tsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16000 entries, 0 to 15999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   파일      16000 non-null  object
 1   종       16000 non-null  int64 
 2   병       16000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 375.1+ KB


In [ ]:
train_tsv['종병'] = train_tsv['종'].astype(str) + '_' + train_tsv['병'].astype(str)

In [ ]:
test_label = dict(zip(train_tsv['종병'].unique(), range(0, 20)))
test_label

{'10_20': 10,
 '11_14': 11,
 '13_1': 12,
 '13_15': 15,
 '13_16': 16,
 '13_17': 17,
 '13_18': 18,
 '13_20': 19,
 '13_6': 13,
 '13_9': 14,
 '3_20': 1,
 '3_5': 0,
 '4_11': 4,
 '4_2': 2,
 '4_7': 3,
 '5_8': 5,
 '7_1': 6,
 '7_20': 7,
 '8_6': 8,
 '8_9': 9}

In [ ]:
unique_label = dict(zip(train_tsv['종병'].unique(), range(0, 20)))
train_tsv['라벨'] = train_tsv['종병'].map(unique_label)

In [ ]:
print(unique_label)

{'3_5': 0, '3_20': 1, '4_2': 2, '4_7': 3, '4_11': 4, '5_8': 5, '7_1': 6, '7_20': 7, '8_6': 8, '8_9': 9, '10_20': 10, '11_14': 11, '13_1': 12, '13_6': 13, '13_9': 14, '13_15': 15, '13_16': 16, '13_17': 17, '13_18': 18, '13_20': 19}


# Image Data Cleaning

In [ ]:
class LeafDataset(Dataset):
  def __init__(self, file_dir, transform, dataframe = None):
    self.file_dir = file_dir
    self.transform = transform
    if self.file_dir == 'train':
      self.labels = dict(zip(dataframe['종병'].unique(), range(0, 20))) # dictionary of unique labels
    self.dataset = []
    self.preprocess()

  def preprocess(self):
    file_list = os.listdir(self.file_dir)
    
    if self.file_dir == 'train':
      file_list.remove('train.tsv')
    elif self.file_dir == 'test':
      file_list.remove('test.tsv')
    print(file_list)

    for name in file_list:
      data = os.path.join(self.file_dir, name)
      
      if self.file_dir == 'train':
        classifier = name.split('_')[0] + '_' + name.split('_')[1]
        label = self.labels[classifier]
        self.dataset.append([data, label])
      
      elif self.file_dir == 'test':
        self.dataset.append(data)
    
    print(f'{self.file_dir} processing finished')

  def __getitem__(self, idx):
    if self.file_dir == 'train':
      filename, label = self.dataset[idx]
      image = Image.open(filename)
      image = self.transform(image)
      return image, label
    
    elif self.file_dir == 'test':
      filename = self.dataset[idx]
      image = Image.open(filename)
      image = self.transform(image)
      return image, filename

  def __len__(self):
    return len(self.dataset)

In [ ]:
transform_train = T.Compose([# T.RandomCrop(size = 256, padding = 32),
                             T.RandomHorizontalFlip(), 
                             T.RandomVerticalFlip(),
                             T.ToTensor(),]) 
                            #  T.Normalize(mean = (0.46340314, 0.48881906, 0.40603018), std = (0.2039205 , 0.18547723, 0.21734475))])
transform_test = T.Compose([T.ToTensor(),]) 
                            # T.Normalize(mean = (0.46340314, 0.48881906, 0.40603018), std = (0.2039205 , 0.18547723, 0.21734475))])

Image Augmentation 과정에서 이미 데이터가 0-1의 값으로 조정이 되어 추가적인 정규화를 진행하지 않았습니다

In [ ]:
train_dataset = LeafDataset('train', transform = transform_train, dataframe = train_tsv)

from torch.utils.data import random_split
train_dataset, val_dataset = random_split(train_dataset, [13000, 3000])
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True, num_workers = 8, )
val_loader = DataLoader(val_dataset, batch_size = 64, shuffle = False, num_workers = 8)

test_dataset = LeafDataset('test', transform = transform_test)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False, num_workers = 8)

In [ ]:
# batch dimension - [64, 3, 256, 256]
for i, data in enumerate(train_loader):
  # print(data)
  print(data[0].shape)
  print(len(data[1]))

In [ ]:
a = []
for i, data in enumerate(train_loader):
  a.append(np.array(data[1]))

In [ ]:
print(len(np.concatenate(a, axis = 0)))
np.unique(np.concatenate(a, axis = 0), return_counts = True)

16000


(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800, 800,
        800, 800, 800, 800, 800, 800, 800]))

## Side note: Calculating Normalization Parameters

In [ ]:
pop_mean = []
pop_std1 = []
pop_std2 = []
for i, data in enumerate(train_loader, 0):
  # print(data[0].shape) # [batch = 256, dimension = 3, 256x256 image]
  if i % 10 == 0:
    print(f'{i} Step Processing')
  numpy_image = data[0].numpy()
  batch_mean = np.mean(numpy_image, axis = (0, 2, 3))
  batch_std1 = np.std(numpy_image, axis = (0, 2, 3))
  batch_std2 = np.std(numpy_image, axis = (0, 2, 3), ddof = 1)
  
  pop_mean.append(batch_mean)
  pop_std1.append(batch_std1)
  pop_std2.append(batch_std2)

pop_mean = np.array(pop_mean).mean(axis = 0)
pop_std1 = np.array(pop_std1).mean(axis = 0)
pop_std2 = np.array(pop_std2).mean(axis = 0)

0 Step Processing
10 Step Processing
20 Step Processing
30 Step Processing
40 Step Processing
50 Step Processing
60 Step Processing


In [ ]:
pop_mean, pop_std1, pop_std2

(array([0.4632882 , 0.48873115, 0.4060708 ], dtype=float32),
 array([0.2038785 , 0.18545303, 0.21732107], dtype=float32),
 array([0.20387852, 0.18545303, 0.21732107], dtype=float32))

# VGG16

In [ ]:
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
# device

device(type='xla', index=1)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
learning_rate = 0.0001
num_epochs = 200
best_acc = 0
model = torchvision.models.vgg19_bn().to(device)
# model.to(device)

criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer = optimizer, milestones = [33, 67], gamma = 0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer = optimizer, mode = 'min', factor = 0.1, patience = 5, verbose = True)

total_loss = [] # list to keep record of loss for each epoch
predictions = {}
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs): # repeat for number of epochs
    model.train() # train model
    # print(optimizer)
    for i, batch in enumerate(train_loader): # for each image in train set
        images, labels = batch[0], batch[1] # assign image torch to images and the label of the image to labels
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)

        loss = criterion(outputs, labels) # compare prediction on each train image with actual label

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer, barrier=True)

        total_loss.append(loss.item())

    print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {sum(total_loss)/len(total_loss):.4f}')
    # scheduler.step() # update learning rate at specific epoch

    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for i, batch in enumerate(val_loader):
            images, labels = batch[0], batch[1]
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            # predictions[name] = predicted
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(correct, total)
        val_acc = 100 * correct / total
        print('Test Accuracy of the model on the test images: {} %'.format(val_acc))

        if val_acc > best_acc:
            best_acc = val_acc
            print('save_model!!!')
            torch.save(model.state_dict(), 'VGG19.pth')

        if epoch % 10 == 0:
            print(outputs)
            print(predicted)
    
    # print('save_model!!!')
    # torch.save(model.state_dict(), 'VGG19.pth')


    scheduler.step(val_acc)

In [ ]:
# device = xm.xla_device()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = torchvision.models.vgg19_bn()
model.load_state_dict(torch.load('VGG19.pth'))
model = model.to(device)

predictions = {}

model.eval()
with torch.no_grad():
  for i, batch in enumerate(test_loader):
      print(i)
      images, name = batch[0], batch[1]
      images = images.to(device)
      
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)

      predictions[name] = predicted
      print(predictions)

In [ ]:
df_list = []
for key in predictions:
  df_list.append(pd.DataFrame([key, list(predictions[key])]).T)

In [ ]:
example = pd.concat(df_list, axis = 0, ignore_index = True, join = 'outer')
example[0] = example[0].str.replace('test/', '')

In [ ]:
example[1] = example[1].map(lambda x: x.item())
example.sort_values(0)

,0,1
973,0.jpg,0
983,1.jpg,1
992,10.jpg,11
864,100.jpg,0
1992,1000.jpg,0
...,...,...
12,995.jpg,16
1974,996.jpg,17
1978,997.jpg,18
1980,998.jpg,19


# Submission

In [ ]:
submission = pd.read_csv('test/test.tsv', header = None)

In [ ]:
submission

,0
0,0.jpg
1,1.jpg
2,2.jpg
3,3.jpg
4,4.jpg
...,...
3992,3992.jpg
3993,3993.jpg
3994,3994.jpg
3995,3995.jpg


In [ ]:
reverse_label = {}
for key in test_label:
  reverse_label[test_label[key]] = key

reverse_label

{0: '3_5',
 1: '3_20',
 2: '4_2',
 3: '4_7',
 4: '4_11',
 5: '5_8',
 6: '7_1',
 7: '7_20',
 8: '8_6',
 9: '8_9',
 10: '10_20',
 11: '11_14',
 12: '13_1',
 13: '13_6',
 14: '13_9',
 15: '13_15',
 16: '13_16',
 17: '13_17',
 18: '13_18',
 19: '13_20'}

In [ ]:
new_submission = submission.merge(example, how = 'outer', copy = True)
new_submission[1] = new_submission[1].map(reverse_label)
new_submission['leaf'] = new_submission[1].map(lambda x: x.split('_')[0])
new_submission['disease'] = new_submission[1].map(lambda x: x.split('_')[1])
new_submission = new_submission.drop(1, axis = 1)
new_submission

,0,leaf,disease
0,0.jpg,3,5
1,1.jpg,3,20
2,2.jpg,4,2
3,3.jpg,4,7
4,4.jpg,4,11
...,...,...,...
3992,3992.jpg,13,15
3993,3993.jpg,13,16
3994,3994.jpg,13,17
3995,3995.jpg,13,18


In [ ]:
new_submission.to_csv('new_submission_.tsv', sep = '\t', index = False, header = False)

In [ ]:
pd.read_csv('new_submission.tsv', sep = '\t', header = None)

,0,1,2
0,0.jpg,3,5
1,1.jpg,3,20
2,2.jpg,4,11
3,3.jpg,4,7
4,4.jpg,4,11
...,...,...,...
3992,3992.jpg,13,9
3993,3993.jpg,13,16
3994,3994.jpg,13,17
3995,3995.jpg,13,18
